In [ ]:
#### RUNNING ACE, PF, BMI ANALYSIS

In [ ]:
#Import Data
import os
import pandas as pd
import numpy as np 
#!pip install nbformat
from sklearn.preprocessing import StandardScaler
#take out max, just do mean for protect score
import seaborn as sns
import matplotlib.pyplot as plt
from pymer4.models import Lmer


In [ ]:
#*****KEYy*****
## sex - male (1) female (2) intersex (3)
## PDS - scores 
## puberty - 1 (yes) 0 (no) or 2 NA or 3 intersex
## BMI - (weight-lb x 703) / (height-in x height)
## ACEs 1 = Physical Abuse; 2 - Sexual Abuse; 3 - Neglect; 4 - Emotional; 5 - Family drug/substance; 6 - Family mental; 7- DV;\
    #8-family structure; 9- death; 10 - neighborhood safety; 11- incarceration; 12 - bullying/discrim; 13- housing/food insecurity
        ## 1 = any positive, 0 = negative, 2 = dont have all results (but changed this to zero)
## race - 0 - NA; 1 - white; 2 - black; 3 - AIAN; 4 - NHPI; 5- asian; 6 - mixed
## ethnicity 1 - hispanic; 2 - non hispanic; 0 - na/refuse to anser 
##protect protect	

In [ ]:
%run #data

# merge demographic and ACE dataframes

In [ ]:
df = Demo_df.merge(ACE_df, on=["src_subject_id", "eventname"], how="outer")

In [ ]:
# Filter the DataFrame for 'baseline_year_1_arm_1' event - creates a dictionary 
tmp = df[df['eventname'] == 'baseline_year_1_arm_1']

# Create a dictionary mapping 'src_subject_id' to 'demo_sex_v2' 
subj_list = dict(zip(tmp['src_subject_id'], tmp['demo_ethn_v2']))

# Map the 'demo_sex_v2' values to the 'sex' column for the entire DataFrame based on 'src_subject_id'
df['demo_ethn_v2'] = df['src_subject_id'].map(subj_list)


In [ ]:
df = df_copy.copy()

# Step 1: Create a mapping of src_subject_id to rel_family_id for baseline rows
baseline_mapping = df[df['eventname'] == 'baseline_year_1_arm_1'].set_index('src_subject_id')['rel_family_id']

# Step 2: Map the rel_family_id to all rows using the src_subject_id
df['rel_family_id'] = df['src_subject_id'].map(baseline_mapping)

In [ ]:
flags = ['height_flagged' , 'height_decrease_event' , 'low_weight_1' , 'low_weight_2', 'low_weight_3' , \
         'weight_loss_flagged' , 'remote' , 'date_flagged' , 'med_flag']

In [ ]:
df.eventname.value_counts()

In [ ]:
df = df[df['height_flagged'] == False]
# df = df[df['height_flagged'] != True]
df.eventname.value_counts()


In [ ]:
# Filter the DataFrame based on the conditions
df = df[(df['height_decrease_event'].isnull()) | (df['eventname'] == '4_year_follow_up_y_arm_1')]
##can keep those with innacuracy year 4 only because its either innacurate year 3 or year 4 
# Verify the remaining event counts
print(df['eventname'].value_counts())


In [ ]:
df = df[df['weight_loss_flagged'] == False]


df = df[df['low_weight_1'] == False]
df = df[df['low_weight_2'] == False]
df = df[df['low_weight_3'] == False]
print(df.eventname.value_counts())

In [ ]:

df = df[df['remote'] == 1]
print(df.eventname.value_counts())
df = df[df['med_flag'] == False]
print(df.eventname.value_counts())
df = df[df['date_flagged'] == False]
print(df.eventname.value_counts())

In [ ]:
bmi_min = df['BMI_v'].min()
bmi_max = df['BMI_v'].max()
print(f'The range of BMI is from {bmi_min} to {bmi_max}') 

In [ ]:
# Calculate the mean and standard deviation of BMI
mean_bmi = df['BMI_v'].mean()
std_bmi = df['BMI_v'].std()


In [ ]:
# Filter the DataFrame for BMI > 50 or BMI < 10
bmi_outliers = df[(df['BMI_v'] > 45) | (df['BMI_v'] < 10)]

# Display the rows with these BMI outliers
print(bmi_outliers[['src_subject_id', 'BMI_v', 'eventname' , 'height', 'weight' , 'sex']])

# display only the unique subject IDs
unique_outlier_ids = bmi_outliers['src_subject_id'].unique()
print("\nUnique subject IDs with BMI > 45 or BMI < 10:")
print(unique_outlier_ids)

In [ ]:
# Lists of subjects for each event year

# Create a flag for abnormal BMI
df['abnormal_bmi_flag'] = np.where(
    (df['src_subject_id'].isin(baseline_subjects)) & (df['eventname'] == 'baseline_year_1_arm_1'),
    1,  # Flag for abnormal BMI (1 indicates to remove)
    np.where(
        (df['src_subject_id'].isin(year1_subjects)) & (df['eventname'] == '1_year_follow_up_y_arm_1'),
        1,  # Flag for abnormal BMI (1 indicates to remove)
        np.where(
            (df['src_subject_id'].isin(year2_subjects)) & (df['eventname'] == '2_year_follow_up_y_arm_1'),
            1,  # Flag for abnormal BMI (1 indicates to remove)
            0  # No flag (0 indicates valid data)
        )
    )
)

# Count the rows where the abnormal_bmi_flag is set to 1 (indicating they are flagged for removal)
rows_to_remove = df[df['abnormal_bmi_flag'] == 1]

# Print the number of rows that will be removed
print(f'Number of rows to be removed: {len(rows_to_remove)}')

# display the rows that are flagged for removal (if you want to review them before removal)
print(rows_to_remove[['src_subject_id', 'eventname', 'height', 'weight', 'BMI_v', 'abnormal_bmi_flag']])


In [ ]:
# Remove rows where the abnormal BMI flag is set to 1
df = df[df['abnormal_bmi_flag'] == 0]

# Get value counts for the cleaned DataFrame (you can change the column name based on what you need)
value_counts = df['src_subject_id'].value_counts()

# Print the value counts
print(value_counts)

event_counts = df['eventname'].value_counts()
print(event_counts)


# now that I have QC'ed data, restrict to year 2 because this is where I will be working

In [ ]:
protect_vars2 = ['protect_parent_recent',
'protect_friend_recent', 'protect_friend_recent2', 
 'protect_self_recent' ]


In [ ]:
# Step 1: Filter for year1 and year2 data
tmp_year1 = df[df['eventname'] == '1_year_follow_up_y_arm_1']
tmp_year2 = df[df['eventname'] == '2_year_follow_up_y_arm_1']

# Step 2: Create mappings for each variable in protect_vars2 for both year1 and year2
for var in protect_vars2:
    # Map year2 data
    year2_mapping = dict(zip(tmp_year2['src_subject_id'], tmp_year2[var]))
    # Map year1 data
    year1_mapping = dict(zip(tmp_year1['src_subject_id'], tmp_year1[var]))

    # Overwrite the column with year2 data, falling back to year1 data
    df[var] = df['src_subject_id'].map(year2_mapping).fillna(df['src_subject_id'].map(year1_mapping))

# Step 3: Display results
print(df[['src_subject_id', 'eventname'] + protect_vars2].head(20))


In [ ]:
df = df[df['eventname'] == '2_year_follow_up_y_arm_1' ]

df = df[df['BMI_v'].notnull()]

In [ ]:
df.rename(columns={'BMI_v': 'BMI'}, inplace=True)


In [ ]:
df.rename(columns={'income_comb_cat': 'income'}, inplace=True)

In [ ]:
# Set a seed for reproducibility
np.random.seed(42)  # can choose any integer value

# Step 1: Shuffle the indices
idx = np.random.permutation(np.arange(len(df)))

# Step 2: Create a copy of the DataFrame and shuffle it
d = df.copy()
d = d.set_index(idx).sort_index()

# Step 3: Drop duplicates based on 'rel_family_id', keeping the first occurrence
d = d.drop_duplicates(subset='rel_family_id', keep='first')

# Step 4: Create a list of 'src_subject_id' to keep
subj_list = d['src_subject_id'].to_list()

# Step 5: Flag records to keep or exclude
df['remove_sibs'] = np.where(df['src_subject_id'].isin(subj_list), 'keep', 'exclude')

# Step 6: Verify the counts
print(df.groupby('eventname')['remove_sibs'].value_counts())

## actually remove siblings

In [ ]:
df = df[df['remove_sibs'] == 'keep']

In [ ]:
df.eventname.value_counts()

In [ ]:
len(df.rel_family_id.unique())

In [ ]:
df = df[df['demo_ethn_v2'] <3]

In [ ]:
df.eventname.value_counts()

In [ ]:
df = df[df['highest_edu'] >0]

In [ ]:
df.eventname.value_counts()

In [ ]:
ace_columns = ['ACE_PA_count', 'ACE_SA_count', 'ACE_N_count', 'ACE_EA_count', 'ACE_D_count', 'ACE_MH_count', 
             'ACE_DV_count',   'ACE_F_count' , 'ACE_NV_count', 'ACE_J2_count', 'ACE_B_count', 'ACE_SS_count']
#'ACE_count'(ACE categories)
df['ACE_total'] = df[ace_columns].sum(axis=1)

In [ ]:
# List of ACE variables to calculate value counts for
ace_columns_3 = ['ACE_PA', 'ACE_SA', 'ACE_N', 'ACE_EA', 'ACE_D', 'ACE_MH', 
                 'ACE_DV', 'ACE_F', 'ACE_NV', 'ACE_J2', 'ACE_B', 'ACE_SS']

# Filter the DataFrame for the specific event ('2_year_follow_up_y_arm_1')
filtered_df = df[df['eventname'] == '2_year_follow_up_y_arm_1']

# Apply a function to count the number of '3's across the specified ACE columns for each subject
filtered_df['total_3s'] = filtered_df[ace_columns_3].apply(lambda row: (row == 3).sum(), axis=1)

# Find how many unique subject IDs have more than 1 '3' across all ACE columns for this specific event
subjects_with_more_than_1_3 = filtered_df[filtered_df['total_3s'] > 1]['src_subject_id'].unique()

# Print the results
print(f"Number of unique subject IDs with more than 1 '3' across all ACE columns (for 2_year_follow_up_y_arm_1 event): {len(subjects_with_more_than_1_3)}")
print(f"Subject IDs with more than 1 '3' across all ACE columns (for 2_year_follow_up_y_arm_1 event): {subjects_with_more_than_1_3}")


In [ ]:
##REMOVE THOSE IN ACE_total MISSING MORE THAN 1 answer 
# List of unique subject IDs with more than 1 '3' in the ACE columns
subjects_with_more_than_1_3 = filtered_df[filtered_df['total_3s'] > 1]['src_subject_id'].unique()

# Remove the rows where 'src_subject_id' is in the list of subjects with more than 1 '3'
df= df[~df['src_subject_id'].isin(subjects_with_more_than_1_3)]

# Print the shape of the new DataFrame to confirm the number of rows removed
print(f"Original DataFrame shape: {df.shape}")
print(f"Cleaned DataFrame shape: {df.shape}")


In [ ]:
# make a simple df

data_df = df[['src_subject_id', 'eventname', 'BMI', 'ACE_total', 'sex', 'interview_age', 'site_id_l',
              'Puberty_PDS', 'highest_edu', 'ACE_sum',  'race', 'income' ,
               'demo_ethn_v2', 'ACE_PA_sum_count', 
              'ACE_SA_sum_count', 'ACE_N_sum_count', 'ACE_EA_sum_count', 'ACE_D_sum_count', 
              'ACE_MH_sum_count', 'ACE_DV_sum_count', 'ACE_F_sum_count', 'ACE_NV_sum_count', 
              'ACE_J2_sum_count', 'ACE_B_sum_count', 'ACE_SS_sum_count',
              'ACE_PA_count', 'ACE_SA_count', 'ACE_N_count', 'ACE_EA_count', 'ACE_D_count', 
              'ACE_MH_count', 'ACE_DV_count',  'ACE_F_count' ,'ACE_NV_count', 
              'ACE_J2_count', 'ACE_B_count', 'ACE_SS_count', 'protect_parent_y2' , 'protect_friend_y2' ,'protect_self_y2','bmiz','bmipct','weight_status']]
#'ACE_FD_count', 'ACE_FD_sum_count',


## standardize the continuous data

In [ ]:
# List of columns to standardize
columns_to_standardize = ['protect_mean_y2','protect_parent_y2' , 'protect_self_y2' , 'protect_friend_y2']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the specified columns, replacing them with their standardized values
data_df[columns_to_standardize] = scaler.fit_transform(data_df[columns_to_standardize])
#tmp1 is the first one to run with just standardiz ace_tota

In [ ]:
####PROTECT MEAN

# List of columns to calculate the average
columns_to_avg = ['protect_self_y2', 'protect_friend_y2', 'protect_parent_y2']

# Create a new column 'average_protection' with the average of the specified columns
data_df['protect_mean_y2'] = data_df[columns_to_avg].mean(axis=1)

In [ ]:
# List of columns to standardize
columns_to_standardize = ['ACE_total', 'interview_age','Puberty_PDS','protect_mean_y2','protect_parent_y2' , 'protect_self_y2' , 'protect_friend_y2' ]

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the specified columns, replacing them with their standardized values
data_df[columns_to_standardize] = scaler.fit_transform(data_df[columns_to_standardize])
#tmp1 is the first one to run with just standardiz ace_tota

## RUN ANALYSES: BMI*ACE

In [ ]:
##BMI~ACE
for v in protect_vars:
    formula = 'BMI ~ ACE_total + sex + interview_age + Puberty_PDS + highest_edu + (1|site_id_l)'
    
    model = Lmer(formula, data=data_df)
    model_fit = model.fit(factors={
        'sex': {"1.0": -1, "2.0": 1},
        'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
    })
    display(model_fit)

In [ ]:
##BMI~ACE~ethnicity
for v in protect_vars:
    formula = 'BMI ~ ACE_total *demo_ethn_v2 + sex + interview_age + Puberty_PDS + highest_edu + (1|site_id_l)'
    
    model = Lmer(formula, data=data_df)
    model_fit = model.fit(factors={
        'sex': {"1.0": -1, "2.0": 1},
        'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
    })
    display(model_fit)

In [ ]:
# Define the unique ethnicities
ethnicities = data_df['demo_ethn_v2'].unique()

# Loop through each ethnicity and fit the model
for ethnicity in ethnicities:
    print(f"Running model for ethnicity: {ethnicity}")
    
    formula = f'BMI ~ ACE_total + sex + interview_age + Puberty_PDS + highest_edu + (1|site_id_l)'
    
    # Fit the model using a condition within the data
    model = Lmer(formula, data=data_df[data_df['demo_ethn_v2'] == ethnicity])
    
    # Fit the model with factors
    model_fit = model.fit(factors={
        'sex': {"1.0": -1, "2.0": 1},
        'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
    })
    
    # Display the results
    print(f"Results for ethnicity: {ethnicity}")
    display(model_fit)


## PROTECT Factors

In [ ]:
#BMI ~ ACE*PF
protect_vars = ['protect_self_y2', 'protect_friend_y2' , 'protect_parent_y2', 'protect_mean_y2']

for v in protect_vars:
    formula = 'BMI ~ ACE_total*' + v + ' + sex + interview_age + Puberty_PDS + highest_edu + (1|site_id_l)'
    
    model = Lmer(formula, data=data_df)
    model_fit = model.fit(factors={
        'sex': {"1.0": -1, "2.0": 1},
        'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
    })
    display(model_fit)

In [ ]:
#BMI ~ ACE*PF*ethnicity
protect_vars = ['protect_self_y2', 'protect_friend_y2' , 'protect_parent_y2','protect_mean_y2']

for v in protect_vars:
    formula = 'BMI ~ ACE_total*' + v + ' *demo_ethn_v2 + sex + interview_age + Puberty_PDS + highest_edu + (1|site_id_l)'
    
    model = Lmer(formula, data=data_df)
    model_fit = model.fit(factors={
        'sex': {"1.0": -1, "2.0": 1},
        'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
    })
    display(model_fit)

In [ ]:
# Define the protective variables and unique ethnicities
protect_vars = ['protect_self_y2', 'protect_friend_y2' , 'protect_parent_y2','protect_mean_y2']
ethnicities = data_df['demo_ethn_v2'].unique()

# Loop through each ethnicity and protective variable
for ethnicity in ethnicities:
    for v in protect_vars:
        print(f"Running model for ethnicity: {ethnicity}, protective variable: {v}")
        
        formula = f'BMI ~ ACE_total * {v} + sex + interview_age + Puberty_PDS + highest_edu + (1|site_id_l)'
        
        # Fit the model for the specific ethnicity
        model = Lmer(formula, data=data_df[data_df['demo_ethn_v2'] == ethnicity])
        model_fit = model.fit(factors={
            'sex': {"1.0": -1, "2.0": 1},
            'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
        })
        
        # Display the results
        print(f"Results for ethnicity: {ethnicity}, protective variable: {v}")
        display(model_fit)


## individual aces

In [ ]:
# Define ACE columns and variables of interest
ace_columns = ['ACE_PA_count', 'ACE_SA_count', 'ACE_N_count', 'ACE_EA_count', 'ACE_D_count', 'ACE_MH_count', 
               'ACE_DV_count', 'ACE_F_count','ACE_NV_count', 'ACE_J2_count', 'ACE_B_count', 
               'ACE_SS_count']
#'ACE_FD_count' 
vars_interest = ['protect_self_y2', 'protect_friend_y2', 'protect_parent_y2', 'protect_mean_y2']

# Nested loop to iterate through protective variables and ACE columns
for var in vars_interest:
    for ace in ace_columns:
        # Print the variable and ACE being analyzed for tracking
        print(f"Running model for ACE: {ace}, Protective Factor: {var}")
        
        # Define the model
        model = Lmer(
            f"BMI ~ {ace} * {var} + interview_age + highest_edu + sex + Puberty_PDS + (1|site_id_l)", 
            data=data_df
        )
        
        # Fit the model with factors
        model_fit = model.fit(factors={
            'sex': {"1.0": -1, "2.0": 1},
            'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
        })
        
        # Display the fitted model results
        display(model_fit)


##ethnicity 

In [ ]:
# Define ACE columns and variables of interest
#ethnicity = 1 hispanic
# Nested loop to iterate through protective variables and ACE columns
for var in vars_interest:
    for ace in ace_columns:
        # Filter the data for ethnicity = 1
        filtered_data = data_df[data_df['demo_ethn_v2'] == 1]
        
        # Print the variable and ACE being analyzed for tracking
        print(f"Running model for ACE: {ace}, Protective Factor: {var}, Ethnicity: 1")
        
        # Define the model
        model = Lmer(
            f"BMI ~ {ace} * {var} + interview_age + highest_edu + sex + Puberty_PDS + (1|site_id_l)", 
            data=filtered_data
        )
        
        # Fit the model with factors
        model_fit = model.fit(factors={
            'sex': {"1.0": -1, "2.0": 1},
            'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
        })
        
        # Display the fitted model results
        display(model_fit)


In [ ]:
#non-hispanic
for var in vars_interest:
    for ace in ace_columns:
        filtered_data = data_df[data_df['demo_ethn_v2'] == 2]
        
        # Print the variable and ACE being analyzed for tracking
        print(f"Running model for ACE: {ace}, Protective Factor: {var}, Ethnicity: 2")
        
        # Define the model
        model = Lmer(
            f"BMI ~ {ace} * {var} + interview_age + highest_edu + sex + Puberty_PDS + (1|site_id_l)", 
            data=filtered_data
        )
        
        # Fit the model with factors
        model_fit = model.fit(factors={
            'sex': {"1.0": -1, "2.0": 1},
            'highest_edu': {"5.0": -1, "4.0": 0.25, "3.0": 0.25, "2.0": 0.25, "1.0": 0.25}
        })
        
        # Display the fitted model results
        display(model_fit)
